In [23]:
from datetime import timedelta, datetime
from django.conf import settings
import os
import glob
from time import mktime
import logging
logger = logging.getLogger(__name__)

def timestamp(delta, fallback):
    """Calculates seconds since epoch of current time minus delta.
    If delta is out of range, calculates timestamp of fallback datetime instead.
    """
    try:
        dt = datetime.now() - delta
    except (OverflowError):
        # logger.exception('locals:%s' % locals())
        dt = fallback
    return int(mktime(dt.timetuple()))

def new_staging_files(staging_subdirectory, fileglob='*.*', min_age=timedelta.min, max_age=timedelta.max):
    """Check for new or updated files in staging area."""
    os.chdir(os.path.join(settings.STAGING_ROOT, staging_subdirectory))
    all_files = glob.glob(fileglob)
    min_mtime = timestamp(min_age, fallback=datetime.max)
    max_mtime = timestamp(max_age, fallback=datetime.min)
    files = [file for file in all_files 
             if min_mtime > os.path.getmtime(file) > max_mtime]
    return sorted(files)

def new_staging_images(**kwargs):
    """Check for new or updated images in staging area"""
    arguments = dict(
        staging_subdirectory='IMAGES',
        fileglob='*.jpg',
    )
    arguments.update(kwargs)
    return new_staging_files(**arguments)

def new_staging_byline_images(**kwargs):
    """Check for new or updated byline images in staging area"""
    arguments = dict(
        staging_subdirectory='BYLINE',
        fileglob='*.jpg',
        max_age=timedelta(days=1),
    )
    arguments.update(kwargs)
    return new_staging_files(**arguments)

def new_staging_pdf_files(**kwargs):
    """Check for new or updated pdf files in staging area"""
    arguments = dict(
        staging_subdirectory='PDF',
        fileglob='UNI1*VER*000.pdf',
        max_age=timedelta(days=100),
    )
    arguments.update(kwargs)
    return new_staging_files(**arguments)

print( '\n'.join(new_staging_images()) )

02-nyh-ekspertutvalg-01-HDH.jpg
02-nyh-ekspertutvalg-02-HDH.jpg
02-nyh-ekspertutvalg-04-HDH.jpg
03-nyh-nyliste-05-HDH.jpg
06-NYH-nyhnsovalg-03-AN.jpg
06-NYH-nyhnsovalg-04-AN.jpg
07-NYH-nyhmedisinst-01-NWL.jpg
07-NYH-nyhukasstude-01-HDH.jpg
07-NYH-nyhukasstude-02-HDH.jpg
07-NYH-nyhukasstude-03-HDH.jpg
0dc35bec9ee7f44f1cb1bc9d03322f77.jpg
1.jpg
1024px-H-gskolen-i-Nesna004.jpg
103655.jpg
11922036-904052146341372-1544195096-o.jpg
11924637-10153531778185097-672597393-o.jpg
13946354779-c85920ad9b-k.jpg
14-NYH-nyhutmelding-03-HDH.jpg
14-NYH-nyhutmelding-04-HDH.jpg
14963524267-d0b53231c2-n.jpg
14963524267-e52b0d894c-h.jpg
14SKN0fa.jpg
15-NYH-nyhmiljouioe-01-HJK.jpg
15-NYH-nyhmiljouioe-02-HJK.jpg
15-NYH-nyhmiljouioe-03-HJK.jpg
15-NYH-nyhmiljouioe-04-HJK.jpg
15-NYH-nyhmiljouioe-05-HJK.jpg
15203740844-5f7e09098c-z.jpg
15562448100-8163fee134-z.jpg
15637918789-881d1ff7b9-z.jpg
16559662270-6637507e60-z.jpg
17-NYH-nyhsvindel-01-HDH.jpg
17-NYH-nyhsvindel-02-HDH.jpg
17-NYH-nyhsvindel-03-HDH.jpg
18-OMV-

In [52]:
files = new_staging_images(max_age=timedelta(days=100))[:50]
file_patterns = ('/' + ImageFile.slugify(fname) for fname in files)
for pattern in file_patterns:
    matches = ImageFile.objects.filter(source_file__endswith=pattern)
    print(pattern, ' '.join(str(match.source_file) for match in matches))
    

/02-nyh-ekspertutvalg-01-HDH.jpg 2015/02/02-nyh-ekspertutvalg-01-HDH.jpg
/02-nyh-ekspertutvalg-02-HDH.jpg 2015/22/02-nyh-ekspertutvalg-02-HDH.jpg
/02-nyh-ekspertutvalg-04-HDH.jpg 2015/09/02-nyh-ekspertutvalg-04-HDH.jpg
/03-nyh-nyliste-05-HDH.jpg 2015/03/03-nyh-nyliste-05-HDH.jpg
/06-NYH-nyhnsovalg-03-AN.jpg 2015/12/06-NYH-nyhnsovalg-03-AN.jpg
/06-NYH-nyhnsovalg-04-AN.jpg 
/07-NYH-nyhmedisinst-01-NWL.jpg 2015/07/07-NYH-nyhmedisinst-01-NWL.jpg
/07-NYH-nyhukasstude-01-HDH.jpg 
/07-NYH-nyhukasstude-02-HDH.jpg 2015/23/07-NYH-nyhukasstude-02-HDH.jpg
/07-NYH-nyhukasstude-03-HDH.jpg 2015/07/07-NYH-nyhukasstude-03-HDH.jpg
/0dc35bec9ee7f44f1cb1bc9d03322f77.jpg 
/1.jpg 2014/16/1.jpg nett/1.jpg
/1024px-H-gskolen-i-Nesna004.jpg 
/103655.jpg 
/11922036-904052146341372-1544195096-o.jpg 
/11924637-10153531778185097-672597393-o.jpg 2015/21/11924637-10153531778185097-672597393-o.jpg
/13946354779-c85920ad9b-k.jpg 
/14-NYH-nyhutmelding-03-HDH.jpg 
/14-NYH-nyhutmelding-04-HDH.jpg 2015/14/14-NYH-nyhutmeldin

In [47]:
def size(fname):
    return int(os.stat(fname).st_size)

def mtime(fname):
    return datetime.fromtimestamp(os.stat(fname).st_mtime)

def ctime(fname):
    return datetime.fromtimestamp(os.stat(fname).st_ctime)

def hashfile(fname, algorithm=hashlib.md5, blocksize=65536):
    """Hexadecimal hash of a file"""
    hasher = algorithm()
    with open(fname, 'rb') as source:
        buffer = source.read(blocksize)
        while len(buffer) > 0:
            hasher.update(buffer)
            buffer = source.read(blocksize)
    return hasher.hexdigest()

def stats(file):
    print('filename: {file}\nmtime:    {mtime}\nctime:    {ctime}\nmd5:      {md5}'.format(
        file=file, mtime=mtime(file), ctime=ctime(file), size=size(file), md5=hashfile(file)))

stats(file)

filename: 21-DEB-debbrei-12-PSS.jpg
mtime:    2015-08-21 15:27:05
ctime:    2015-10-16 23:24:37.027950
md5:      98e89e5dc7d328dfcbb2d69633195d05


In [5]:
{datetime.now():'now'}

{datetime.datetime(2015, 10, 22, 20, 47, 9, 422159): 'now'}

In [6]:
datetime.min.strftime('%s')

'-62135599380'

In [9]:
import os
from datetime import datetime

now = datetime.now()
IMAGE_STAGING = os.path.join(settings.STAGING_ROOT, 'IMAGES')
os.chdir(IMAGE_STAGING)
print(os.path.abspath(os.curdir))

/srv/fotoarkiv_universitas/STAGING/IMAGES


In [11]:
for file_path in sorted(os.listdir(), key=lambda f: os.path.getmtime(f), reverse=True)[:10]: 
    mtime = os.path.getmtime(file_path)
    modified = datetime.fromtimestamp(mtime)
    print('{0:<40.40}  {1}'.format(file_path, modified))
    

Nummer-14-onsdag-18-oktober1989.jpg       2015-10-14 11:32:00
28-SID-oyeblikk-03-AOB.jpg                2015-10-14 11:31:58
statsbudsjett.jpg                         2015-10-14 11:31:58
28-SID-oyeblikk-02-AOB.jpg                2015-10-14 11:31:57
28-SID-oyeblikk-01-AOB.jpg                2015-10-14 11:31:56
28-NYH-nyhrektorkje-03-XX.jpg             2015-10-14 11:31:54
rektor plektor fra _SLEKTOR_.JPEG         2015-10-14 11:31:54
Arkivfoto-uio-rektor-KjetilBlom.jpg       2015-10-14 11:31:54
28-NYH-nyhrektorkje-01-XX.jpg             2015-10-14 11:31:52
26-NYH-nyharbeidsmo-03-HB.jpg             2015-10-14 11:31:52


In [12]:
ImageFile.objects.filter(source_file__icontains=file_path)

[]

In [13]:
file_path

'26-NYH-nyharbeidsmo-03-HB.jpg'

In [17]:
ImageFile.objects.filter(source_file__icontains='26-NYH-nyha')

[]

In [53]:
ImageFile.objects.filter(source_file__size__gt=10**6)

FieldError: Unsupported lookup 'size' for ImageField or join on the field not permitted.